In [124]:
'''
This code outputs the energy loss rates following Barnes&Kasen2016
(ApJ 829:110, 2016)
Section 3: Thermalisation physics.
Processes by which energetic decay products thermalize in the KN ejecta.
Focus on beta-particles.
(Taken from ./energyLossrates.py)
'''

import numpy as np
import matplotlib.pyplot as plt

# Constants
lambda_ee_ = 10  # Coulomb logarithm for electron-electron scattering
m_e_ = 9.10938356e-28  # electron mass (g)
m_p_ = 1.6726219e-24 # proton mass (g)
r_e_ = 2.8179403227e-13  # classical electron radius (cm)
m_u_ = 1.660539e-24  # nuclear mass unit (g)
c_ = 2.99792458e10  # light speed (cm/s)
kB_ = 8.617333262145e-11  # Boltzmann constant (MeV/K)
Msun_ = 2.0e33 # Solar mass (g)

delta_ = 1  # power-law profile for inner ejecta (Barnes&Kasen2016)
n_ = 1  # power-law profile for outer ejecta (Barnes&Kasen2016)


def to_erg(E):
    return(E*1.60218e-6)


def to_MeV(E):
    return(E/1.60218e-6)


def plasma_losses(E_beta, n_e, T):
    ''' Returns beta_particle losses from interaction with free thermal
    electrons (MeV/s).
    args:
    E_beta = beta-particle kinetic energy (MeV)
    n_e = free electron number density (cm-3)
    T = ejecta temperature (MeV)
    '''

    return(7.7e-15 
           * E_beta**(-.5) 
           * n_e * lambda_ee_ 
           * (1. - (3.9 / 7.7) * T / E_beta))


def calc_n_eb(rho, z_over_a):
    ''' Returns the bound electron number density (cm-3) for a given
    average Z/A.
    args:
    rho = mass density (g/cm-3)
    z_over_a = composition (0.4 inner / 0.55 outer ejecta)
    '''
    return(rho / m_u_ * z_over_a)


def IE_losses(E_beta, Ibar, v_beta, n_eb):
    ''' Returns beta_particle losses due to ionization and excitation
    of atomic electrons (MeV/s).
    args:
    E_beta = beta-particle kinetic energy (MeV)
    I_bar = average ionization and excitation potential (MeV) (eq. 9)
    v_beta = beta-particle speed (cm/s)
    n_eb = bound electron number density (cm-3) 
    '''

    tau = to_erg(E_beta) / (m_e_ * c_**2)  # E_beta converted to erg
    g = 1 + tau**2 / 8 - (2*tau+1) * np.log(2)
    Edot_IE = 2*np.pi * r_e_**2 * m_e_ * c_**3 * n_eb / (v_beta/c_)
    Edot_IE *= (2 * np.log(E_beta/Ibar) 
                + np.log(1 + tau/2) 
                + (1 - (v_beta/c_)**2) * g)

    return(Edot_IE / 1.60218e-6)


def synch_losses(v_beta, B):
    ''' Returns synchrotron losses from Barnes&Kasen2016 eq. 11. (erg/s)
    args:
    v_beta = beta particle velocity
    B = Magnetic field strength
    '''
    gamma = np.sqrt(1./(1-(v_beta/c_)**2))
    loss = (4./9.) * r_e_**2 * c_ * gamma**2 * (v_beta / c_)**2 * B**2
    return(loss)


def calc_mass_density(r, t, E, Mej, delta=delta_, n=n_):
    ''' Returns the mass density for a given r and t in the ejecta.
    args:
    r = radius (cm)
    t = time (s)
    E = Explosion energy (erg)
    M = ejecta mass (Msun)
    '''

    zeta_v = 1
    zeta_rho = 1

    v_t = 7.1e8 * zeta_v * ((E*1.e-51) / (Mej/Msun))**(.5)

    v = r/t
    if v < v_t:
        rho = zeta_rho * (Mej/(v_t*t)**3) \
              * (r / (v_t*t))**(-delta)
    else:
        rho = zeta_rho * (Mej/(v_t*t)**3) \
              * (r / (v_t*t))**(-delta)
    return(rho)


In [125]:
def plot_rates():
    plt.plot(E_beta, plasma_losses(E_beta, n_e, T) / rho, label='plasma')
    plt.plot(E_beta, IE_losses(E_beta, Ibar, v_beta, n_eb) / rho, label='IE')
#     [plt.plot(E_beta, IE_losses(E_beta, Ibar, v_beta, n_eb) / rho) for alpha in np.logspace(-5,5,10)]
    # plt.plot(E_beta, synch_losses(v_beta, B) / rho)
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()

### Plotting loss rates for typical values

Ejecta mass: $M_\mathrm{ej} = 10^{-3} M_\odot$ \
Ejecta velocity: $v_\mathrm{ej} = 0.2c$

Nebular phase starts at $t \sim 100$ days:

Ejecta density = $M_\mathrm{ej} / \frac{4}{3} \pi (v_\mathrm{ej}t)^3 = 9 \times 10^{12}$ g.cm$^{-2}$


In [126]:
M_ej = 5.e-3 * Msun_
v_ej = 0.2 * c_
t = 1 * 86400.
rho_ej = M_ej / ((4./3.) * np.pi * (v_ej * t)**3)
print("%e" %rho_ej)

1.717189e-14


In [129]:
E_beta = np.logspace(-2, 1, 100) # particle kinetic energy (MeV)
T = 1e-6 # MeV
z_over_a = 0.4
n_e = rho_ej / m_p_ *1.e-3  # number density of bound electrons
    # elements heavier than H are singly ionized (low temperatures)
    # Composition from Barnes+2016 fig.1 (took A_heavy=1.e-3)
    # (this can be refined)
Ibar = np.exp(6.4) * 1.e-6 # (MeV)
gamma_beta = 1 + to_erg(E_beta) / (m_e_ * c_**2)
v_beta = np.sqrt(1 - 1./gamma_beta**2) * c_
rho = rho_ej
# B = 3.7e-6

n_eb = calc_n_eb(rho, z_over_a)

%matplotlib widget
plot_rates()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
v_beta/c_

array([0.19498581, 0.20169937, 0.20862885, 0.21577956, 0.22315675,
       0.23076562, 0.23861125, 0.24669858, 0.25503242, 0.26361734,
       0.27245769, 0.28155756, 0.29092068, 0.30055041, 0.31044968,
       0.32062095, 0.33106609, 0.34178638, 0.35278241, 0.36405401,
       0.37560017, 0.38741897, 0.39950751, 0.41186181, 0.42447673,
       0.4373459 , 0.45046162, 0.46381483, 0.47739499, 0.49119001,
       0.50518627, 0.51936848, 0.53371969, 0.5482213 , 0.56285301,
       0.57759285, 0.59241726, 0.60730109, 0.62221776, 0.63713931,
       0.65203659, 0.66687942, 0.68163679, 0.69627708, 0.7107683 ,
       0.72507842, 0.73917556, 0.7530284 , 0.76660641, 0.77988023,
       0.79282192, 0.8054053 , 0.81760622, 0.82940281, 0.84077573,
       0.85170834, 0.86218684, 0.87220042, 0.88174128, 0.89080464,
       0.89938876, 0.90749478, 0.91512665, 0.92229095, 0.9289967 ,
       0.93525511, 0.9410794 , 0.94648445, 0.95148662, 0.95610341,
       0.96035323, 0.96425514, 0.9678286 , 0.97109323, 0.97406

In [99]:
IE_losses(E_beta, Ibar, v_beta, n_eb)

array([-3.44462200e+24, -3.27207098e+24, -3.10746764e+24, -2.95047049e+24,
       -2.80075195e+24, -2.65799770e+24, -2.52190621e+24, -2.39218815e+24,
       -2.26856592e+24, -2.15077315e+24, -2.03855424e+24, -1.93166387e+24,
       -1.82986663e+24, -1.73293653e+24, -1.64065665e+24, -1.55281872e+24,
       -1.46922273e+24, -1.38967662e+24, -1.31399590e+24, -1.24200329e+24,
       -1.17352844e+24, -1.10840759e+24, -1.04648330e+24, -9.87604115e+23,
       -9.31624333e+23, -8.78403715e+23, -8.27807240e+23, -7.79704856e+23,
       -7.33971251e+23, -6.90485629e+23, -6.49131502e+23, -6.09796493e+23,
       -5.72372142e+23, -5.36753738e+23, -5.02840152e+23, -4.70533689e+23,
       -4.39739946e+23, -4.10367689e+23, -3.82328741e+23, -3.55537883e+23,
       -3.29912766e+23, -3.05373840e+23, -2.81844295e+23, -2.59250013e+23,
       -2.37519530e+23, -2.16584016e+23, -1.96377259e+23, -1.76835663e+23,
       -1.57898248e+23, -1.39506663e+23, -1.21605199e+23, -1.04140809e+23,
       -8.70631234e+22, -